# Creación de capas personalizadas

En ciertas ocasiones es posible que se desee implementar una nueva capa de la red neuronal artificial para la que no existe una implementación en Tensorflow o Keras. En estos casos podemos utilizar Tensorflow para crear capas personalizadas.

### Importando el conjunto de datos

In [1]:
from tensorflow.keras import datasets

In [2]:
boston_housing = datasets.boston_housing

In [3]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [4]:
import pandas as pd
import numpy as np

features = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

df_train = pd.DataFrame(np.column_stack([X_train, y_train]), columns=features)
df_train.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,15.2
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,42.3
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,17.7
5,0.28392,0.0,7.38,0.0,0.493,5.708,74.3,4.7211,5.0,287.0,19.6,391.13,11.74,18.5
6,9.18702,0.0,18.10,0.0,0.700,5.536,100.0,1.5804,24.0,666.0,20.2,396.90,23.60,11.3
7,4.09740,0.0,19.58,0.0,0.871,5.468,100.0,1.4118,5.0,403.0,14.7,396.90,26.42,15.6
8,2.15505,0.0,19.58,0.0,0.871,5.628,100.0,1.5166,5.0,403.0,14.7,169.27,16.65,15.6
9,1.62864,0.0,21.89,0.0,0.624,5.019,100.0,1.4394,4.0,437.0,21.2,396.90,34.41,14.4


### División del conjunto de datos

In [5]:
from sklearn.model_selection import train_test_split

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

### Escalando el conjunto de datos

In [6]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train_prep = scaler.fit_transform(X_train)
X_val_prep = scaler.transform(X_val)
X_test_prep = scaler.transform(X_test)

### Definicion de la arquitectura de la Red Neuronal Artificial

### Creación de capas sin parámetros

Las capas sin parámetros aplican transformaciones sobre los valores de entrada. Algunos ejemplos de capas de este tipo que se encuentran implementados en Keras son: _keras.layers.Flatten_ o _keras.layers.ReLU_. 

En estos casos, la mejor forma de crear este tipo de capas es desarrollar una función personalizada y utilizarla junto con la construcción _keras.layers.Lambda_.

In [7]:
from tensorflow import keras
import tensorflow as tf

# Capa personalizada que eleva al cuadrado los valores de entrada
square_layer = keras.layers.Lambda(lambda x: tf.square(x))
# esta capa no tiene parametros asociados

Esta capa se utiliza de la misma forma que el resto de capas que se encuentran implementadas en Keras por defecto

In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [10]:
network = models.Sequential()

network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(square_layer) # Capa personalizada
network.add(layers.Dense(1))

In [11]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                420       
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 lambda (Lambda)             (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 741
Trainable params: 741
Non-trainable params: 0
_________________________________________________________________


In [12]:
network.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae'])

In [13]:
history = network.fit(X_train_prep, 
                      y_train, 
                      epochs=50, 
                      validation_data=(X_val_prep, y_val))

Epoch 1/50
13/13 [==============================] - 3s 16ms/step - loss: 582.3309 - mae: 21.8147 - val_loss: 575.2356 - val_mae: 22.0508
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 563.0556 - mae: 21.4965 - val_loss: 566.6235 - val_mae: 21.8942
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 546.6823 - mae: 21.2167 - val_loss: 550.2344 - val_mae: 21.6030
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 525.2801 - mae: 20.8001 - val_loss: 528.3524 - val_mae: 21.1515
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 500.5154 - mae: 20.3551 - val_loss: 494.9253 - val_mae: 20.3954
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 464.3133 - mae: 19.5478 - val_loss: 454.3465 - val_mae: 19.2297
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 421.3266 - mae: 18.4276 - val_loss: 411.8301 - val_mae: 17.8649
Epoch 8/50
13/13 [=============================

In [14]:
network.predict(X_test_prep)

array([[49.282413 ],
       [19.168009 ],
       [12.83803  ],
       [23.009192 ],
       [18.358524 ],
       [20.33453  ],
       [ 7.0382185],
       [14.808978 ],
       [20.844635 ],
       [36.791924 ],
       [36.517635 ],
       [20.713589 ],
       [27.470465 ],
       [17.480545 ],
       [24.050426 ],
       [29.664713 ],
       [20.037489 ],
       [29.823442 ],
       [26.22793  ],
       [26.505526 ],
       [15.71492  ],
       [17.5526   ],
       [33.889317 ],
       [ 8.94143  ],
       [23.846813 ],
       [28.355175 ],
       [21.0189   ],
       [23.500391 ],
       [36.627876 ],
       [33.255943 ],
       [39.472816 ],
       [14.771689 ],
       [22.658653 ],
       [14.724661 ],
       [24.662813 ],
       [22.964836 ],
       [22.965197 ],
       [13.948857 ],
       [20.333986 ],
       [17.863558 ],
       [34.437206 ],
       [23.7695   ],
       [21.077942 ],
       [10.747956 ],
       [18.031816 ],
       [40.66752  ],
       [24.347637 ],
       [18.30